In [1]:
import os

HOME = os.path.expanduser('~')

DATA_FOLDER = os.path.abspath(
    os.path.join(HOME, 'projects', 'cshl-singlecell-2017', 'data'))
FIGURE_FOLDER = os.path.abspath(
    os.path.join(HOME, 'projects', 'cshl-singlecell-2017', 'figures'))


notebook_name = '50_Example_workflow_reanalyzing_macosko2015'
data_folder = os.path.join(DATA_FOLDER, notebook_name)
figure_folder = os.path.join(FIGURE_FOLDER, notebook_name)

input_folder = os.path.join(DATA_FOLDER, '91_filter_genes')

! mkdir -p $data_folder
! mkdir -p $figure_folder

In [3]:
from sklearn.decomposition import PCA, FastICA, NMF
from sklearn.manifold import TSNE, MDS

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import phenograph

import macosko2015
%matplotlib inline

In [5]:
counts, cells, genes = macosko2015.load_differential_clusters()
print('counts.shape', counts.shape)
print('cells.shape', cells.shape)
print('genes.shape', genes.shape)

counts.shape (6020, 1339)
cells.shape (6020, 5)
genes.shape (1339, 39)


In [6]:
counts.head()

,1500015O10RIK,1500016L03RIK,1700025G04RIK,1810009A15RIK,1810037I17RIK,2010107E04RIK,2410066E13RIK,2610017I09RIK,2900011O08RIK,4632428N05RIK,...,ZFHX4,ZFP36,ZFP365,ZFP36L1,ZFP804A,ZIC1,ZIC4,ZMAT4,ZWINT,ZYX
r1_GGCCGCAGTCCG,0,0,5,4,7,28,5,0,33,0,...,3,0,7,0,0,0,0,7,53,0
r1_CTTGTGCGGGAA,0,0,9,4,4,33,4,0,43,0,...,8,0,4,0,0,0,0,5,65,1
r1_GCGCAACTGCTC,0,0,11,1,4,26,2,0,30,0,...,0,0,2,0,0,0,0,4,38,0
r1_GATTGGGAGGCA,0,0,8,2,5,14,6,0,20,0,...,1,0,1,0,0,1,0,3,17,0
r1_GTGCCGCCTCTC,0,0,1,19,1,13,0,0,0,0,...,0,0,3,0,0,0,0,0,3,0


In [8]:
genes.head()

,cluster_01,cluster_02,cluster_03,cluster_04,cluster_05,cluster_06,cluster_07,cluster_08,cluster_09,cluster_10,...,cluster_30,cluster_31,cluster_32,cluster_33,cluster_34,cluster_35,cluster_36,cluster_37,cluster_38,cluster_39
1500015O10RIK,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1500016L03RIK,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1700025G04RIK,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1810009A15RIK,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1810037I17RIK,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [9]:
cells.head()

,cluster_id,celltype,cluster_n,cluster_n_celltype,cluster_celltype_with_id
r1_GGCCGCAGTCCG,cluster_02,Retinal ganglion cells,2,#2 (Retinal ganglion cells),Retinal ganglion cells (cluster_02)
r1_CTTGTGCGGGAA,cluster_02,Retinal ganglion cells,2,#2 (Retinal ganglion cells),Retinal ganglion cells (cluster_02)
r1_GCGCAACTGCTC,cluster_02,Retinal ganglion cells,2,#2 (Retinal ganglion cells),Retinal ganglion cells (cluster_02)
r1_GATTGGGAGGCA,cluster_02,Retinal ganglion cells,2,#2 (Retinal ganglion cells),Retinal ganglion cells (cluster_02)
r1_GTGCCGCCTCTC,cluster_25,Cones,25,#25 (Cones),Cones (cluster_25)
